In [49]:
import pandas as pd
import datetime as dt


# Data Request for March 22, 2025
# How many clients received Case Management with SALT after a service with SALT?

beginDate = dt.datetime(2020, 1, 1)
endDate = dt.datetime(2025, 4, 1)

# Read data
dfServices = pd.read_csv('All SALT Services.csv')
dfServices['Service Date'] = pd.to_datetime(dfServices['Service Date'])

# Make a list of only case management services
dfServicesCopy = dfServices.copy()
dfSuccess = dfServicesCopy[dfServicesCopy['Service'].str.contains('Case Management')]

# Make a list of non case management services
dfServices = dfServices[~dfServices['Service'].str.contains('Case Management')]

dfServicesInTimeframe = dfServices[(dfServices['Service Date'] >= beginDate) & (dfServices['Service Date'] <= endDate)]
dfSuccessInTimeframe = dfSuccess[(dfSuccess['Service Date'] >= beginDate) & (dfSuccess['Service Date'] <= endDate)]

##### Successful Transitions Report #####
print(f"-------- Successful Transitions Report --------")
print(f"Date Range: {beginDate.strftime('%B %d %Y')} to {endDate.strftime('%B %d %Y')}")
print('\n')

# Total unduplicated clients served
total_clients_served = dfServicesInTimeframe['Client ID'].nunique()
print(f"Total unduplicated clients served with SALT, not including Case Management: {total_clients_served}")

# Total clients that recieved case management
total_clients_served = dfSuccessInTimeframe['Client ID'].nunique()
print(f"Total number of unduplicated clients that recieved Case Management services with SALT: {total_clients_served}")
print('\n')

# After recieving a SALT service, how many clients sought out case management?
dfSaltServices1 = dfServicesInTimeframe.copy()
dfSaltServices1 = dfSaltServices1.sort_values('Service Date').drop_duplicates('Client ID', keep='first')

# Store when case management was attended
dfCaseManagement1 = dfSuccess.copy()
dfCaseManagement1 = dfCaseManagement1.sort_values('Service Date').drop_duplicates('Client ID', keep='last')
dfCaseManagement1['CM Service Date'] = dfCaseManagement1['Service Date']
dfCaseManagement1 = dfCaseManagement1.drop(columns=['Service Date'])

# Merge non CM services with CM services to compare timelines
# 'Service Date' will be NaN on every CM service
dfMerge1 = pd.merge(dfSaltServices1, dfCaseManagement1, on='Client ID', how='inner')
dfMerge1 = dfMerge1[dfMerge1['Service Date'] < dfMerge1['CM Service Date']].reset_index(drop=True)

# Print data
print(len(dfMerge1), 'people received a SALT Case Management session after receiving a SALT service') 
print(round(len(dfMerge1)/len(dfSaltServices1)*100, 2), '% of people who received a SALT service received a Case Management service')
print('\n')

/var/folders/nm/sgb0qrh5259964c0_5jt6zfh0000gn/T/ipykernel_21593/2870361264.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dfServices['Service Date'] = pd.to_datetime(dfServices['Service Date'])


-------- Successful Transitions Report --------
Date Range: January 01 2020 to April 01 2025


Total unduplicated clients served with SALT, not including Case Management: 5341
Total number of unduplicated clients that recieved Case Management services with SALT: 1127


797 people received a SALT Case Management session after receiving a SALT service
14.92 % of people who received a SALT service received a Case Management service


